In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import sys
sys.path.insert(1, '../../rtsvg')
from rtsvg import *
rt = RACETrack()
df = pd.read_csv('llama3_8b_2014_vast_sbs.csv')
len(df), df['llama3_8b_time'].sum()/60.0/60.0 # appears to be about 8 hours...

In [ ]:
def listEscapes(s):
    _ret_ = []
    i = 0
    while i < len(s):
        if s[i] == '\\': _ret_.append(s[i:i+4])
        i += 1
    return _ret_
def fixEscapes(s):
    _escapes_ = listEscapes(s)
    _ret_ = s
    for i in range(len(_escapes_)):
        if len(_escapes_[i]) != 4 or _escapes_[i][1] != 'x': continue
        _ret_ = _ret_.replace(_escapes_[i], chr(int(_escapes_[i][2:],16)))
    return _ret_

_jsons_ = []
_failed_parse_counts_, _failed_parses_, _successful_parse_counts_, _ignored_counts_ = 0, [], 0, 0
for _json_str_ in df['llama3_8b_response']:
    if _json_str_.startswith("I cannot") or _json_str_.startswith("I apologize"):
        _ignored_counts_ += 1
        continue
    _json_str_ = fixEscapes(_json_str_)
    if '{' not in _json_str_ or '}' not in _json_str_: 
        _failed_parse_counts_ += 1
        _failed_parses_.append(_json_str_)
        continue
    _json_str_ = _json_str_[_json_str_.index('{'):_json_str_.rindex('}')+1]
    _json_     = None
    try:
        _json_ = json.loads(_json_str_)
        _successful_parse_counts_ += 1
    except:
        try:
            _json_ = json.loads(_json_str_ + '}')
            _successful_parse_counts_ += 1
        except:
            try:
                _json_ = json.loads(_json_str_ + '] }')
                _successful_parse_counts_ += 1
            except:
                try:
                    _json_ = json.loads('[' + _json_str_ + ']')
                    _successful_parse_counts_ += 1                
                except:
                    _failed_parse_counts_ += 1
                    _failed_parses_.append(_json_str_)
    if _json_ is not None: _jsons_.append(_json_)
_successful_parse_counts_ ,_failed_parse_counts_

In [ ]:
#j = 0
#_splits_ = fixEscapes(_failed_parses_[j]).split('\n')
#for i in range(len(_splits_)):
#    print(f'{i:-3} {_splits_[i]}')
#json.loads(fixEscapes(_failed_parses_[j]) + '] }')

In [ ]:
#_set_ = set()
#for i in range(len(_failed_parses_)):
#    _escapes_ = listEscapes(_failed_parses_[i])
#    _set_ = _set_ | set(_escapes_)
#_set_

In [ ]:
_pos_ = {}
from os.path import exists
_json_repr_ = rt.jsonRepr(_jsons_)
_df_, _relates_, _labels_ = _json_repr_.starPathGraphDataFrame()
_df_ = pl.from_pandas(_df_)
_rtg_ = rt.interactiveGraphLayout(_df_, {'relationships':_relates_, 'node_labels':_labels_, 'link_size':'vary', 'pos':_pos_, 'link_opacity':0.2}, w=1900, h=1000)
if exists('llama3_8b_sbs_layout.csv'): _rtg_.loadLayout('llama3_8b_sbs_layout.csv')
_rtg_

In [ ]:
#_rtg_.saveLayout('llama3_8b_sbs_layout.csv')

In [ ]:
_pos_ = _rtg_.pos